## 데이터 솎아내기

In [1]:
import json 
import os
import pandas as pd
from pycocotools.coco import COCO

In [2]:
coco = COCO('../../dataset/train.json')

train_df = pd.DataFrame()

image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
id = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    ann_ids = coco.getAnnIds(imgIds=image_info['id'])
    anns = coco.loadAnns(ann_ids)
    file_name = image_info['file_name']
        
    for ann in anns:
        image_ids.append(file_name)
        class_name.append(classes[ann['category_id']])
        class_id.append(ann['category_id'])
        x_min.append(float(ann['bbox'][0]))
        y_min.append(float(ann['bbox'][1]))
        x_max.append(float(ann['bbox'][0]) + float(ann['bbox'][2]))
        y_max.append(float(ann['bbox'][1]) + float(ann['bbox'][3]))
        id.append(coco.loadImgs(image_id)[0]['id'])        

train_df['image_id'] = image_ids
train_df['class_name'] = class_name
train_df['class_id'] = class_id
train_df['x_min'] = x_min
train_df['y_min'] = y_min
train_df['x_max'] = x_max
train_df['y_max'] = y_max
train_df['id'] = id



loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [3]:
train_df

,image_id,class_name,class_id,x_min,y_min,x_max,y_max,id
0,train/0000.jpg,General trash,0,197.6,193.7,745.4,663.4,0
1,train/0001.jpg,Metal,3,0.0,407.4,57.6,588.0,1
2,train/0001.jpg,Plastic bag,7,0.0,455.6,144.6,637.2,1
3,train/0001.jpg,Glass,4,722.3,313.4,996.6,565.3,1
4,train/0001.jpg,Plastic,5,353.2,671.0,586.9,774.4,1
...,...,...,...,...,...,...,...,...
23139,train/4882.jpg,Plastic,5,0.0,116.2,944.1,930.3,4882
23140,train/4882.jpg,Plastic bag,7,302.1,439.3,567.3,655.4,4882
23141,train/4882.jpg,General trash,0,511.3,451.1,570.0,481.3,4882
23142,train/4882.jpg,Paper,1,255.0,421.4,526.7,616.5,4882


In [4]:
z_score_limit = 6 # sigma 범위
remove_df = pd.DataFrame()

index_ = pd.DataFrame()
mean_value = train_df['id'].value_counts().mean()
std_value = train_df['id'].value_counts().std()

index_['z_score'] = (train_df['id'].value_counts()-mean_value)/std_value
removed_name = [i for i in index_.index if index_['z_score'][i] > z_score_limit]
#removed_id = [ i for i in removed_index if ]


In [5]:
train_df['id'].value_counts()

4197    71
946     55
4047    52
394     48
2981    46
        ..
3359     1
3357     1
1605     1
1608     1
0        1
Name: id, Length: 4883, dtype: int64

In [6]:
removed_name.sort()
removed_name

[394, 946, 1678, 1955, 2416, 2981, 4047, 4197]

In [7]:
len(removed_name)

8

In [8]:
train_df['id'].value_counts()[len(removed_name):]

4492    42
1797    42
4841    41
4778    40
3825    40
        ..
3359     1
3357     1
1605     1
1608     1
0        1
Name: id, Length: 4875, dtype: int64

## updated_train.json 생성

In [9]:
# 새 파일 생성
root = '../../dataset/'
file_name = 'train.json'
save_name = 'changed_train.json'
train_df = pd.DataFrame

with open(os.path.join(root,file_name),'r') as f:
    coco_json = json.load(f)


In [10]:
coco_json.keys()

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

In [11]:
coco_json['images'][61:64]  # image 원본 데이터 id = 62번 데이터 존재.

[{'width': 1024,
  'height': 1024,
  'file_name': 'train/0061.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-27 16:38:42',
  'id': 61},
 {'width': 1024,
  'height': 1024,
  'file_name': 'train/0062.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-01-04 17:20:05',
  'id': 62},
 {'width': 1024,
  'height': 1024,
  'file_name': 'train/0063.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-26 14:32:24',
  'id': 63}]

In [12]:
coco_json['annotations'][290:294] # annotation 원본 데이터 image_id = 62번 데이터 존재.

[{'image_id': 61,
  'category_id': 3,
  'area': 251336.47,
  'bbox': [265.4, 332.9, 535.1, 469.7],
  'iscrowd': 0,
  'id': 290},
 {'image_id': 62,
  'category_id': 7,
  'area': 164740.8,
  'bbox': [308.8, 164.3, 490.3, 336.0],
  'iscrowd': 0,
  'id': 291},
 {'image_id': 62,
  'category_id': 7,
  'area': 98912.58,
  'bbox': [94.5, 337.8, 360.6, 274.3],
  'iscrowd': 0,
  'id': 292},
 {'image_id': 62,
  'category_id': 1,
  'area': 154521.43,
  'bbox': [406.3, 426.2, 490.7, 314.9],
  'iscrowd': 0,
  'id': 293}]

In [13]:
coco_info = coco_json['info']
coco_licenses = coco_json['licenses']
coco_images = [ i for i in coco_json['images'] if i['id'] not in removed_name ]
coco_categories = coco_json['categories']
coco_annotations = [ i for i in coco_json['annotations'] if i['image_id'] not in removed_name ]

In [14]:
coco_images[61:64] # id : 0062.jpg 데이터 누락

[{'width': 1024,
  'height': 1024,
  'file_name': 'train/0061.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-27 16:38:42',
  'id': 61},
 {'width': 1024,
  'height': 1024,
  'file_name': 'train/0062.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2021-01-04 17:20:05',
  'id': 62},
 {'width': 1024,
  'height': 1024,
  'file_name': 'train/0063.jpg',
  'license': 0,
  'flickr_url': None,
  'coco_url': None,
  'date_captured': '2020-12-26 14:32:24',
  'id': 63}]

In [15]:
coco_annotations[290:294] # image_id : 0062.jpg 데이터 누락

[{'image_id': 61,
  'category_id': 3,
  'area': 251336.47,
  'bbox': [265.4, 332.9, 535.1, 469.7],
  'iscrowd': 0,
  'id': 290},
 {'image_id': 62,
  'category_id': 7,
  'area': 164740.8,
  'bbox': [308.8, 164.3, 490.3, 336.0],
  'iscrowd': 0,
  'id': 291},
 {'image_id': 62,
  'category_id': 7,
  'area': 98912.58,
  'bbox': [94.5, 337.8, 360.6, 274.3],
  'iscrowd': 0,
  'id': 292},
 {'image_id': 62,
  'category_id': 1,
  'area': 154521.43,
  'bbox': [406.3, 426.2, 490.7, 314.9],
  'iscrowd': 0,
  'id': 293}]

In [16]:
new_data = {
    'info': coco_info,
    'licenses': coco_licenses,
    'images': coco_images,
    'categories': coco_categories,
    'annotations': coco_annotations
}

In [17]:
new_data.keys()

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])

In [18]:
# 새 파일 생성
root = '../../dataset/'
file_name = 'train.json'
save_name = 'updated_train.json'
train_df = pd.DataFrame
output_path = os.path.join(root,save_name)

with open(output_path,'w') as f:
    json.dump(new_data, f, indent=2)